<a href="https://colab.research.google.com/github/aamatthews/2022UWCourse/blob/main/00_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands on Radar Data, Part 1
### Alyssa Matthews and Ya-Chien Feng

#Setup

Welcome to Google Colab! It allows you to play with notebooks in python, without having to install anything on your own computer. 

To access the notebooks, go to this link in your browser:

https://github.com/aamatthews/UWCourse2022

or

https://bit.ly/3UdCC2a


Before we get started, there is a little setup we have to do first. 

In each notebook we will be using PyART, the Python ARM Radar Toolkit, that is helpful for reading in, plotting, and contains multiple algorithms for digging in deeper to radar data. Since this doesn't come natively in Colab, we must first install it prior to importing. This can take a few minutes, so lets start it running first and we can cover some other basics.

In [ ]:
!pip install arm-pyart

We will start with learning to read in and plot the various types of ARM radar data - Vertically Pointing, RHIs, and PPIs.


*   To run a line, press shift+enter on your keyboard, or click the play button

Let's grab some data for today! Run the below cell, click the folder icon on the left, then click the refresh folder icon to see the data appear. 

In [ ]:
!wget https://engineering.arm.gov/~amatthews/hou_data_sample/20220701.zip

In [ ]:
!unzip 20220701.zip

Next, lets import the libraries we will need in this notebook. We will add more later in the course in future notebooks, but these are the basics I tend to use most often in my work.

In [ ]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import pandas as pd
import pyart

import matplotlib.pyplot as plt
%matplotlib inline

# Reading in files

In this section, I will show you three different ways you can read in files. Which you use really depends on which you prefer or which will work best for your needs.

###Option 1: NetCDF4 library 

For more information, you can view the documentation here:
https://unidata.github.io/netcdf4-python/#Dataset.__init__

In [ ]:
data = nc.Dataset('20220701/houkazrcfrgeM1.a1.20220701.190000.nc')

In the radar files, time is split into base time (the time at the start of the file) and time offset (the time since the file started). I like to save a combined timestamp of the two after reading in the data, as it can make the plots and data easier to interpret later on.

In [ ]:
time =  pd.to_datetime(data.variables['base_time'][:] + data.variables['time_offset'][:], unit='s')

In [ ]:
time[0:10]

Once you have read in the data file, there are various levels you can look at. First, is the data class. This will include all the attributes (or metadata), as well as the variables stored in the file.

In [ ]:
data

Often, it is useful just to see what the variable names are. You can see the full details of each variable with data.variables, but to see just a list of the variable names, you can add the keys() function.

In [ ]:
data.variables.keys()

Once you know the name of a variable you want to look at, you can examine it to see the details of that variable, including a more detailed name, the dimension, and units. Lets take a look at Reflectivity.

In [ ]:
data.variables['reflectivity']

Finally, to view the data within the variable, you can add [:]

In [ ]:
data.variables['reflectivity'][:]

Lets view a piece of the attributes or metadata. These are called directly from the dataset.

In [ ]:
data.antenna_altitude

###Option 2: Xarray library 

This is another library that can read in netCDF4 files. Data read in with this library is in a more pandas-like structure (for those familiar with pandas). You can view the documentation for reading in a file with xarray here: 

https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html#xarray.open_dataset

An ability I particularly like within xarray is its ability to read in multiple data files at once and save them to one structure instead of having to read in each file individually. For this course, though, we will just be focusing on reading in one file at a time for simplicity. The documentation for this can be found at:

https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset

In [ ]:
radar = xr.open_dataset('20220701/houkazrcfrgeM1.a1.20220701.190000.nc', decode_times = False)

One known issue in Xarray is how it handles the 'time' variable in the ARM radar files. To correct this, I overwrite that variable with the combined base_time + time_offset, as I showed previously in the netCDF4 example.

In [ ]:
radar['time'] = pd.to_datetime(radar['base_time'].values + radar['time_offset'].values, unit='s')

To start digging into the data, lets first print the Dataset. As you can see, the output is much more condensed, with many options to expand the data to see what you are interested in. 

In [ ]:
radar

Lets next get a list of the available variable names.

In [ ]:
list(radar.variables.keys())

To see the details of a particular variable, you can either expand it out in the radar output, or can use a command identical to the one we used in the netCDF4 library example. Lets look at Reflectivity again, and at the data within that variable.

In [ ]:

radar.variables['reflectivity']

To view the data within the variable, you can add .data or .values to the end of the variable.

In [ ]:
radar.variables['reflectivity'].data

Viewing an attribute is done similarly to what we learned in the netCDF4 library example.

In [ ]:
radar.antenna_diameter

###Option 3: PyART

The PyART package can read in many different types of radar files. You can view its documentation at: 

https://arm-doe.github.io/pyart/

In [ ]:
radar = pyart.io.read('20220701/houkazrcfrgeM1.a1.20220701.190000.nc')

Lets view the information about this radar object. This will show us the variables and their information at the top, and the attributes and metadata at the bottom.

In [ ]:
radar.info()

One difference between PyART and the netCDF4 or Xarray libraries is that PyART calls the data names 'fields' instead of 'variables'. Lets view them now.

In [ ]:
radar.fields

In [ ]:
radar.fields['reflectivity']

To view the data within the variable, you just add ['data'] to the end of the field.

In [ ]:
radar.fields['reflectivity']['data']

Viewing the attributes or metadata is done a little differently to the other two examples when using PyART

In [ ]:
radar.metadata['antenna_altitude']

##Exercise 1
Pick one of the methods I showed above and read in a XSACR, KASACR, or CSAPR2 data file.

Can you find the scan type for that file in the data?

###Hint

it is an attribute or metadata, and not a variable

:)

###Solutions

In [ ]:
data = xr.open_dataset('20220701/houcsapr2cfrS2.a1.20220701.192028.nc')

In [ ]:
data.scan_name

In [ ]:
data = nc.Dataset('20220701/houcsapr2cfrS2.a1.20220701.192028.nc')

In [ ]:
data.scan_name

In [ ]:
data = pyart.io.read('20220701/houcsapr2cfrS2.a1.20220701.192028.nc')

In [ ]:
data.metadata['scan_name']

#Now, lets plot some data!


The ARM scanning radars have the ability to scan in multiple ways. Here, we will go over various methods for plotting the most common scan types. First I will show a method with no specialized libraries, then I will show a method using PyART.

![radar scan image](https://journals.ametsoc.org/view/journals/atot/31/3/images/full-jtech-d-13-00044_1-f3.jpg)

##Vertically pointing data (VPT)

###Method 1

In [ ]:
kazr = xr.open_dataset('20220701/houkazrcfrgeM1.a1.20220701.190000.nc', decode_times = False)
kazr['time'] = pd.to_datetime(kazr['base_time'].values + kazr['time_offset'].values, unit='s')

In [ ]:
plt.figure(figsize=(15,5))
plt.pcolormesh(kazr['time'], kazr['range'], kazr['reflectivity'].T, cmap = 'jet')
plt.colorbar(label = 'Reflectivity (dBZ)')
plt.xlabel('Time')
plt.ylabel('Height (m)')
plt.title('KAZR Reflectivity');

###Method 2: PyART

In [ ]:
radar = pyart.io.read('/content/20220701/houkazrcfrgeM1.a1.20220701.190000.nc')

In [ ]:

display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[8,4])
# ax = fig.add_subplot(111)
display.plot_vpt('reflectivity', vmin=-20, vmax=40)


##RHI or HSRHI

###Method 1

In [ ]:
csapr = xr.open_dataset('/content/20220701/houcsapr2cfrS2.a1.20220701.192028.nc', decode_times = False)
csapr['time'] = pd.to_datetime(csapr['base_time'].values + csapr['time_offset'].values, unit='s')

In [ ]:
csapr.scan_name

In [ ]:
sweep_start = list(map(int,(csapr['sweep_start_ray_index'].data)))
sweep_end = list(map(int,(csapr['sweep_end_ray_index'].data)))
azi = csapr['azimuth'].data
elv = csapr['elevation'].data
r = csapr['range'].data

y = np.outer(r, np.sin((elv)*np.pi/180.0))
x = np.outer(r, np.cos((elv)*np.pi/180.0))
sweep_num = 0



for i in range(len(sweep_start)):
    sweep_num = i
#     print(sweep_num)
    plt.figure(figsize=(20,40))
    plt.subplot(6,1,1)
    plt.pcolormesh(x[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   y[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   csapr['reflectivity'].data[sweep_start[sweep_num]:sweep_end[sweep_num],:], 
                   vmin=-60, vmax=50, cmap='jet')
    clb = plt.colorbar()
    clb.set_label('Zh')
    plt.xlabel('Horizontal Distance from Radar (m)')
    plt.ylabel('Height (m)')
    azi_start = np.nanmedian(azi[sweep_start[sweep_num]])
    plt.title('Azimuth: %f ' % azi_start)
    plt.ylim(0,20000)


###Method 2: PyART

In [ ]:
radar = pyart.io.read('/content/20220701/houcsapr2cfrS2.a1.20220701.192028.nc')

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[10, 5])
display.plot('reflectivity', 0, vmin=-16, vmax=64.0)
# plt.ylim(0,20)

##PPI





###Method 1

This is very similar to the RHI, except instead of y using azimuth, you use elevation and flip from cos to sin (or vice versa)

In [ ]:
ppi = xr.open_dataset('/content/20220701/houcsapr2cfrS2.a1.20220701.192603.nc', decode_times = False)
ppi['time'] = pd.to_datetime(ppi['base_time'].values + ppi['time_offset'].values, unit='s')

In [ ]:
ppi.scan_name

In [ ]:
sweep_start = list(map(int,(ppi['sweep_start_ray_index'].data)))
sweep_end = list(map(int,(ppi['sweep_end_ray_index'].data)))
azi = ppi['azimuth'].data
elv = ppi['elevation'].data
r = ppi['range'].data

y = np.outer(r, np.cos((azi)*np.pi/180.0))
x = np.outer(r, np.sin((azi)*np.pi/180.0))
sweep_num = 0

for i in range(len(sweep_start)):
    sweep_num = i
#     print(sweep_num)
    plt.figure(figsize=(10,7.5))

    plt.pcolormesh(x[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   y[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   ppi['reflectivity'].data[sweep_start[sweep_num]:sweep_end[sweep_num],:], 
                   vmin=-60, vmax=40, cmap='jet')
    clb = plt.colorbar()
    clb.set_label('Zh')
    plt.xlabel('E-W Distance from Radar (m)')
    plt.ylabel('N-S Distance from Radar (m)')
    elv_start = elv[sweep_start[sweep_num]+50]
    plt.title('Elevation: %f' % elv_start)

###Method 2: PyART

In [ ]:
radar = pyart.io.read('/content/20220701/houcsapr2cfrS2.a1.20220701.192603.nc')

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[7,6])

display.plot('reflectivity', 0, vmin=-32, vmax=64.)
display.plot_range_rings([20, 40, 60, 80, 100])
display.plot_cross_hair(2.)

##Exercise
1. Plot an XSACR or KASACR scanning file. 

2. What do you see in the data?